In [ ]:
# Dependencies
import pandas as pd

### Neighborhood info in the ACS file
http://www.mncompass.org/profiles/neighborhoods/minneapolis-saint-paul

Source: 2014-2018 American Community Survey 5-year estimates, adjusted to fit current neighborhood boundaries using the 2010 Census counts. The 5-year estimates represent averages of data collected over that time period. (courtesy of mncompass.org)

In [ ]:
# Load & Visualize ACS data
ACS = pd.read_csv('resources/acs_2014_2018.csv', encoding='UTF-8')
print(ACS.shape)
ACS.head(1)

In [ ]:
# Reduce columns (Note I did not include margin of error columns)
# *** Folwell has no info
ACS_reduced = ACS[['Neighborhood', 'City', 'Total population',
                         'White - Share',
                         'Of Color - Share',
#                          'Foreign born - Share',
#                          'English only - Share',
#                          'Language other than English - Share',
#                          'Speak English less than "very well" - Share',
#                          'Population with a disability - Share',
                         'Total housing units',
                         'Total households',
                         'Family households - Share',
                         'Married-couple family households - Share',
#                          'Single-person family households - Share',
                         'Nonfamily households - Share',
#                          'Householder living alone - Share',
#                          '65 years and over - Share',
#                          'Households with one or more children under 18 years - Share',
#                          'Households with one or more people 65 years and over',
                         'Vacant housing units - Share',
                         'Occupied housing units - Share',
                         'Average household size (occupied)',
                         'Owner-occupied - Share',
                         'Average owner-occupied household size',
                         'Renter-occupied - Share',
                         'Average renter-occupied household size',
#                          'No vehicles - Share',
#                          'Moved in 2010 or later - Share',
#                          'Moved in 2000 to 2009 - Share',
#                          'Cost-burdened households - Share',
#                          'Cost-burdened owner households - Share',
#                          'Cost-burdened renter households - Share',
#                          'Median rent (2008-2012; 2012 dollars)',
#                          'Median household income (2008-2012; 2012 dollars)',
#                          'Less than $35000 - Share',
#                          '$35000-$49999 - Share',
#                          '$50000-$74999 - Share',
#                          '$75000-$99999 - Share',
#                          '$100000 or more - Share',
#                          'With income below poverty - Share',
#                          'With income 100-149% of poverty - Share',
#                          'With income 150-199% of poverty - Share',
#                          'With income 200% of poverty or higher - Share',
#                          'Proportion of working age adults who are employed - Share',
#                          'Unemployment rate - Share',
#                          'Population (25 years and older) - Share',
#                          'Less than high school - Share',
#                          'High school diploma or GED - Share',
#                          "Some college or associate's degree - Share",
#                          "Bachelor's degree - Share",
#                          'Graduate or professional degree - Share',
#                          'Car; truck; or van (including passengers) - Share',
#                          'Public transportation - Share',
#                          'Walked; biked; worked at home; or other - Share',
#                          'Less than 10 minutes - Share',
#                          '10-19 minutes - Share',
#                          '20-29 minutes - Share',
#                          '30 minutes or longer - Share',
#                          'Population without health insurance coverage - Share',
                         ]]
print(ACS_reduced.shape)
ACS_reduced.head(1)

### Neighborhood info in the Census file
http://www.mncompass.org/profiles/neighborhoods/minneapolis-saint-paul

Source: 2010 Census counts (courtesy of mncompass.org)

In [ ]:
# Load & Visualize Census data
census = pd.read_csv('resources/census_2010.csv', encoding='UTF-8')
print(census.shape)
census.head(1)

In [ ]:
# Reduce columns, matching to ACS dataframe
census_reduced = census[['Neighborhood', 'City', 'Total population',
                         'White - Share',
                         'Of Color - Share',
                         'Total housing units',
                         'Total households',
                         'Family households - Share',
                         'Married-couple family households - Share',
                         'Nonfamily households - Share',
                         'Vacant housing units - Share',
                         'Occupied housing units - Share',
                         'Average household size (occupied)',
                         'Owner-occupied - Share',
                         'Average owner-occupied household size',
                         'Renter-occupied - Share',
                         'Average renter-occupied household size',
                         ]]
print(census_reduced.shape)
census_reduced.head(1)

In [ ]:
# Merge ACS & census files & visualize data
merged_census_mini = pd.merge(ACS_reduced, census_reduced, how = 'outer')
print(merged_census_mini.shape)
merged_census_mini.head(20)

In [ ]:
merged_census_mini.replace('%', '', regex=True, inplace = True)
merged_census_mini.head(1)

In [ ]:
# Save merged file to csv
merged_census_mini.to_csv('resources/merged_census_mini.csv', index=False)

In [ ]:
# Reload csv as it changes all data types to floats
merged_census_mini = pd.read_csv('resources/merged_census_mini.csv', encoding='UTF-8')

### Minneapolis crime data
https://opendata.minneapolismn.gov/datasets/neighborhood-crime-stats

In [ ]:
# Load & Visualize Mpls crime data
mplsCrime = pd.read_csv('resources/crime_mpls.csv', encoding='UTF-8')
print(mplsCrime.shape)
mplsCrime.head(1)

In [ ]:
# Filter out unassigned neighborhoods
mplsCrime = mplsCrime.loc[(mplsCrime['neighborhood'] != '** Not Assigned **') &\
                        (mplsCrime['neighborhood'] != 'Z_** Not Assigned **'), :]

print(mplsCrime.shape)

In [ ]:
# Filter out 2017 incidents
mplsCrime = mplsCrime.loc[(mplsCrime['reportYear'] != 2017)]
print(mplsCrime.shape)

In [ ]:
# Drop NaNs as they represents 0
mplsCrime.dropna(how = 'any', inplace = True)
print(mplsCrime.shape)

In [ ]:
# Filter data with 0 values & convert to integer
mplsCrime = mplsCrime.loc[(mplsCrime['number'] != 0), :]
mplsCrime['number'] = mplsCrime['number'].astype('int64')
print(mplsCrime.shape)

In [ ]:
# Add City column
mplsCrime['City'] = 'Minneapolis'

# Eliminate columns
mplsCrime = mplsCrime[['City', 'neighborhood', 'ucrDescription','reportMonth', 'reportYear', 'number']]

# Rename columns
mplsCrime.columns = ['City', 'Hood', 'Incident', 'Month', 'Year', 'Count']

mplsCrime.head()

In [ ]:
# Update hood names to match other data sources
mplsCrime['Hood'] = mplsCrime['Hood'].str.replace\
                ('Bryn - Mawr', 'Bryn Mawr').str.replace\
                ('Cedar - Isles - Dean', 'Cedar-Isles-Dean').str.replace\
                ('Como', 'Como (Minneapolis)').str.replace\
                ('Ecco', 'ECCO').str.replace\
                ('Lind - Bohanon', 'Lind-Bohanon').str.replace\
                ('Mid - City Industrial', 'Mid-City Industrial').str.replace\
                ('Near - North', 'Near North').str.replace\
                ('Carag', 'South Uptown').str.replace\
                ('Sumner - Glenwood', 'Sumner-Glenwood').str.replace\
                ('University Of Minnesota', 'University of Minnesota').str.replace\
                ('Webber - Camden', 'Webber-Camden').str.replace\
                ('Willard - Hay', 'Willard-Hay').str.replace\
                ('Webber - Camden', 'Webber-Camden')

### St. Paul crime data
https://information.stpaul.gov/Public-Safety/Crime-Incident-Report-Dataset/gppb-g9cg

In [ ]:
# Load & Visualize St. Paul crime data
spCrime = pd.read_csv('resources/crime_st_paul.csv', encoding='UTF-8')
print(spCrime.shape)
spCrime.head(1)

In [ ]:
# Filter out unwanted crimes
spCrime = spCrime.loc[(spCrime['INCIDENT'] == 'Theft') |\
                        (spCrime['INCIDENT'] == 'Auto Theft') |\
                        (spCrime['INCIDENT'] == 'Burglary') |\
                        (spCrime['INCIDENT'] == 'Agg. Assault') |\
                        (spCrime['INCIDENT'] == 'Robbery') |\
                        (spCrime['INCIDENT'] == 'Agg. Assault Dom.') |\
                        (spCrime['INCIDENT'] == 'Arson') |\
                        (spCrime['INCIDENT'] == 'Rape') |\
                        (spCrime['INCIDENT'] == 'Homicide'), :]
print(spCrime.shape)

In [ ]:
# Match remaining crime names to mpls data
spCrime['INCIDENT'] = spCrime['INCIDENT'].str.replace\
                ('Agg. Assault Dom.', 'Aggravated Assault').str.replace\
                ('Agg. Assault', 'Aggravated Assault').str.replace\
                ('Theft', 'Larceny').str.replace\
                ('Auto Larceny', 'Auto Theft')

In [ ]:
# Convert date column
spCrime['DATE'] = pd.to_datetime(spCrime['DATE'])

# Create month & year columns to match mpls data
spCrime['Month'] = spCrime['DATE'].dt.month
spCrime['Year'] = spCrime['DATE'].dt.year

# Filter to 2018 - 2020
spCrime = spCrime.loc[(spCrime['DATE'] >= '01-01-2018') &\
                        (spCrime['DATE'] < '10-1-2020'), :]
print(spCrime.shape)
spCrime.head(2)

In [ ]:
# Add city column
spCrime['City'] = 'St. Paul'

# Eliminate columns & reorganize
spCrime = spCrime[['City', 'NEIGHBORHOOD NAME', 'INCIDENT', 'Month', 'Year', 'Count']]

# Rename columns
spCrime.columns = ['City', 'Hood', 'Incident', 'Month', 'Year', 'Count']

spCrime.head()

In [ ]:
# Parse hood name
spCrime['Hood'] = spCrime['Hood'].str.split(' - ', expand = True)[1]
spCrime.head(2)

In [ ]:
# Update hood names to match other data sources
spCrime['Hood'] = spCrime['Hood'].str.replace('/', '-').str.replace\
                ('Capitol River', 'Downtown').str.replace\
                ('Como', 'Como (St. Paul)').str.replace\
                ('Conway-Battlecreek-Highwood', 'Battle Creek-Highwood').str.replace\
                ('St. Anthony', 'St. Anthony Park').str.replace\
                ('(','/').str.replace(')','')

In [ ]:
# Groupby 
spCrime = spCrime.groupby(['City', 'Hood', 'Year', 'Month', 'Incident'])['Count']\
        .count().to_frame().reset_index()
print(spCrime.shape)
spCrime.head(10)

In [ ]:
# Merge Mpls & St. Paul files & visualize data
merged_crime = pd.merge(mplsCrime, spCrime, how = "outer")
print(merged_crime.shape)
merged_crime.head(1)

In [ ]:
# Save merged file to csv
merged_crime.to_csv('resources/merged_crime.csv', index=False)

In [ ]:
# Reload csv as it changes all data types to floats
merged_crime = pd.read_csv('resources/merged_crime.csv', encoding='UTF-8')
merged_crime.head()

In [ ]:
groupby = merged_crime.groupby(['City', 'Year', 'Incident']).sum()
groupby